# engine

> Simple engine that calculates gradients and backpropagates. Can be used in a simple neural net

In [ ]:
#| default_exp engine

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
import torch
import math

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
class Value:
    def __init__(self,
                 data, # numeric value wrapped by `Value`
                 _children=(), # inputs to a given `Value`
                 _op='', # operation that resulted in a given `Value`
                ):
        self.data = data
        self._prev = set(_children)
        self.op = _op
    
    @staticmethod
    def _wrap(x):
        "Wrap x in Value"
        return x if isinstance(x, Value) else Value(x)
        
    def __add__(self, other):
        "self + other"
        other = self._wrap(other)
        out = Value(self.data + other.data, (self, other), _op='+')
        return out
        
    def __radd__(self, other):
        "other + self"
        return self + other
    
    def __sub__(self, other):
        "self - other"
        return self + (-other)
    
    def __rsub__(self, other):
        "other - self"
        return other + (-self)
    
    def __mul__(self, other):
        "self * other"
        other = self._wrap(other)
        out = Value(self.data * other.data, (self, other), _op='*')
        return out
    
    def __rmul__(self, other):
        "other * mult"
        return self * other
 
    def __pow__(self, other):
        "self ** other. Other should be int or float"
        assert isinstance(other, (int,float)), f"{other} must be int or float"
        out = Value(self.data ** other, (self,), f"**{other}")
        return out
    
    def __truediv__(self, other):
        "self / other"
        return self * other**-1
    
    def __rtruediv__(self, other):
        "other / self"
        return other * self**-1
    
    def __neg__(self):
        "-self"
        return self *-1
    
    def tanh(self):
        "tanh"
        out = Value(math.tanh(self.value), (self,) , 'tanh')
        return out
        
    def relu(self):
        "relu"
        out = Value(max(self.data,0), (self,), 'relu')
        return out
        
    def __repr__(self):
        return f'Value(data={self.data})'

In [ ]:
#Value.relu(b)

In [ ]:
a = Value(-4.0)
b = Value(6.0)

test_eq((a+b).data, -4+6)
test_eq((a-b).data, -4-6)
test_eq((a/b).data, -4/6)
test_eq((a**2).data, (-4)**2)

In [ ]:
show_doc(Value.relu)

---

[source](https://github.com/Iamalos/micrograd-nbdev/blob/main/micrograd_nbdev/engine.py#L73){target="_blank" style="float:right; font-size:smaller"}

### Value.relu

>      Value.relu ()

relu

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()